# Matching Real Reads

### Downloading Fasta file

In [1]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2022-06-20 21:22:57--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.64.171.60, 18.64.171.223, 18.64.171.149, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.64.171.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa’

phix.fa             100%[===================>]   5.40K  --.-KB/s    in 0s      

2022-06-20 21:22:57 (558 MB/s) - ‘phix.fa’ saved [5528/5528]



### Create a function to read a genome from a fasta file

#### (Note: filename=r'filepath' to file since you are passing in a string)

In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()  

    return genome

genome = readGenome(r'phix.fa')
genome[:100]

'GAGTTTTATCGCTTCCATGACGCAGAAGTTAACACTTTCGGATATTTCTGATGAGTCGAAAAATTATCTTGATAAAGCAGGAATTACTACTGCTTGTTTA'

### Create a Function  for the Naive Exact Matching Algorithm

In [3]:
def naive(p, t):
    occurences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if not t[i+j] == p[j]:
                match = False
                break
        if match:
            occurences.append(i)
    return occurences

### Test out the Naive Function

In [4]:
t = 'AGCTTAGATAGC'
p = 'AG'
naive(p, t)

[0, 5, 9]

### Check the result from the Naive Function

In [5]:
t[0:2]

'AG'

In [6]:
t[5:7]

'AG'

In [7]:
t[9:11]

'AG'

### Download a set of sequencing reads from the phix genome

In [8]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2022-06-20 21:26:30--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.64.171.60, 18.64.171.223, 18.64.171.149, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.64.171.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq’

ERR266411_1.first10 100%[===================>] 248.42K  --.-KB/s    in 0.09s   

2022-06-20 21:26:31 (2.60 MB/s) - ‘ERR266411_1.first1000.fastq’ saved [254384/254384]



### Use the readFastQ function to read the downloaded FASTQ file

In [10]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
          fh.readline()
          seq = fh.readline().rstrip()
          fh.readline()
          qual = fh.readline().rstrip()
          if len(seq) == 0:
              break
          sequences.append(seq)
          qualities.append(qual)
    return sequences, qualities

phix_reads, _ = readFastq('ERR266411_1.first1000.fastq')

### Count how many sequence reads have matched the phix genome

In [11]:
numMatched = 0
n = 0
for r in phix_reads:
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
      numMatched += 1

print('%d / %d reads matched the phix genome!' % (numMatched, n))

7 / 1000 reads matched the phix genome!


### Repeat matching using only the first 30 bases of the read
#### (Matched reads obtained above is low)

In [12]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
      numMatched += 1

print('%d / %d reads matched the phix genome!' % (numMatched, n))

459 / 1000 reads matched the phix genome!


### Use the reverse complement function to boost the number of sequence matches to the phix genome.
#### (Obtain both strands of the DNA sequence)

In [15]:
def reverseComplement(s):
    complement = {'A':'T', 'C':'G', 'G':'C', 'T':'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [25]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
      numMatched += 1

print('%d / %d reads matched the phix genome!' % (numMatched, n))

932 / 1000 reads matched the phix genome!
